#### Utilities

In [3]:
!wget https://arxiv.org/pdf/2406.04692
!mv 2406.04692 MoA.pdf

--2024-12-27 13:47:23--  https://arxiv.org/pdf/2406.04692
Resolving arxiv.org (arxiv.org)... 151.101.131.42, 151.101.67.42, 151.101.195.42, ...
Connecting to arxiv.org (arxiv.org)|151.101.131.42|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1157463 (1.1M) [application/pdf]
Saving to: ‘2406.04692’

2406.04692          100%[===================>]   1.10M  2.13MB/s    in 0.5s    

2024-12-27 13:47:24 (2.13 MB/s) - ‘2406.04692’ saved [1157463/1157463]



In [14]:
from dotenv import load_dotenv
from default_prompts import ENGLISH_SYSTEM_PROMPT, VIETNAMESE_SYSTEM_PROMPT
load_dotenv()

True

In [5]:
from pypdf import PdfReader

def get_PDF_text(file : str):
    text = ''

    # Read the PDF file and extract text
    try:
        with open(file, 'rb') as f:
            reader = PdfReader(f)
            text = "\n\n".join([page.extract_text() for page in reader.pages])
    except Exception as e:
        raise f"Error reading the PDF file: {str(e)}"

        # Check if the PDF has more than ~400,000 characters
        # The context lenght limit of the model is 131,072 tokens and thus the text should be less than this limit
        # Assumes that 1 token is approximately 4 characters
    if len(text) > 400000:
        raise "The PDF is too long. Please upload a PDF with fewer than ~131072 tokens."

    return text
  
pdf_text = get_PDF_text('MoA.pdf')

#### LLama with Ollama python

In [10]:
import ollama

response: ollama.ChatResponse = ollama.chat(model='llama3.1',
                                            messages=[{
                                                'role': 'system', 'content': ENGLISH_SYSTEM_PROMPT
                                            }, {
                                                'role': 'user', 'content': pdf_text
                                            }]
                                            )


In [15]:
response.message.content

'This appears to be a research paper on a novel approach to aggregating responses from multiple large language models (LLMs) to produce a single, more accurate and informative response. Here\'s a summary of the key points:\n\n**Approach:**\n\n* The authors propose a method for aggregating responses from multiple LLMs using a MoA (Multi-Objective Attention) layer.\n* The MoA layer takes in responses from different models and outputs a weighted combination of their scores, where each model is given a preference score by the GPT-4 model.\n\n**Case Studies:**\n\n* Two case studies are presented to demonstrate the effectiveness of the approach:\n\t+ Case 1: "Smooth" song facts. The aggregated response from Qwen1.5-110B-Chat achieves a high preference score (0.99) and incorporates key points from other models.\n\t+ Case 2: Authorship question. Although none of the proposed responses achieve a high preference score, the aggregator successfully identifies strong points from each response and p

#### Gemini with SimplerLLM

In [6]:
from SimplerLLM.language.llm import LLM, LLMProvider

gemini = LLM.create(
    provider=LLMProvider.GEMINI,
    model_name='gemini-1.5-flash'
)

gemini_response = gemini.generate_response(
    prompt=pdf_text,
    max_tokens=400000,
    system_prompt=ENGLISH_SYSTEM_PROMPT
)

In [7]:
with open("gemini_response.txt", 'w') as f:
    f.write(gemini_response)

In [8]:
import json 

with open('gemini_response.txt', 'r') as f:
    script_dict = json.load(f)

In [9]:
script_dict['podcast_script']['title']

'Unlocking LLM Potential: The Mixture-of-Agents Approach'

#### Gemini with Vietnamese prompt and Vietnamese text

In [11]:
with open("/home/tminh/Documents/Dataset/thu-vien-phap-luat/json-data/800.json", 'r') as f:
    pdf_vn_text = json.load(f)

law_doc = pdf_vn_text[1]['noi_dung']
law_doc

'ỦY BAN NHÂN DÂN TỈNH LÀO CAI -------- CỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM Độc lập - Tự do - Hạnh phúc --------------- Số: 17/2021/QĐ-UBND Lào Cai, ngày 29 tháng 3 năm 2021 QUYẾT ĐỊNH BAN HÀNH QUY ĐỊNH VỊ TRÍ, CHỨC NĂNG, NHIỆM VỤ, QUYỀN HẠN VÀ CƠ CẤU TỔ CHỨC CỦA CHI CỤC TRỒNG TRỌT VÀ BẢO VỆ THỰC VẬT TỈNH LÀO CAI ỦY BAN NHÂN DÂN TỈNH LÀO CAI Căn cứ Luật Tổ chức chính quyền địa phương ngày 19/6/2015; Căn cứ Luật Sửa đổi, bổ sung một số điều của Luật Tổ chức Chính phủ và Luật Tổ chức chính quyền địa phương ngày 22/11/2019; Căn cứ Luật Ban hành văn bản quy phạm pháp luật ngày 22/6/2015; Căn cứ Luật sửa đổi, bổ sung một số điều của Luật Ban hành văn bản quy phạm pháp luật ngày 18/6/2020; Căn cứ Nghị định số 34/2016/NĐ-CP ngày 14/5/2016 của Chính phủ quy định chi tiết một số điều và biện pháp thi hành Luật Ban hành văn bản quy phạm pháp luật; Căn cứ Nghị định số 154/2020/NĐ-CP ngày 31/12/2020 của Chính phủ Sửa đổi, bổ sung một số điều của Nghị định số 34/2016/NĐ-CP ngày 14/5/2016 của Chính ph

In [15]:
gemini_vn_response = gemini.generate_response(
    prompt=law_doc,
    max_tokens=400000,
    system_prompt=VIETNAMESE_SYSTEM_PROMPT
)
with open("gemini_vn_response.txt", 'w') as f:
    f.write(gemini_vn_response)